## Intialization

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import pickle
import statistics
import pandas as pd

from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import tqdm
import nltk
from tqdm import tqdm
from google.colab import files

In [ ]:
!pip install spacy
!pip install spacy-transformers
!python -m spacy download en_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-04-22 00:28:25.318662: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-22 00:28:26.414784: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.3/460.3 MB 3.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


## GAP Coref Dataset 

In [ ]:
df = pd.read_csv("gap-development.tsv",sep='\t')

In [ ]:
row1 = df.iloc[0]
row1

ID                                                    development-1
Text              Zoe Telford -- played the police officer girlf...
Pronoun                                                         her
Pronoun-offset                                                  274
A                                                    Cheryl Cassidy
A-offset                                                        191
A-coref                                                        True
B                                                           Pauline
B-offset                                                        207
B-coref                                                       False
URL               http://en.wikipedia.org/wiki/List_of_Teachers_...
Name: 0, dtype: object

In [ ]:
example = row1["Text"]

In [ ]:
import spacy
import spacy_transformers
nlp = spacy.load('en_core_web_trf')
doc = nlp(example)

## BERT Self-Attention Visualization
Make sure to enter accuracies.npy and gap-development dataset

In [ ]:
!pip install -q transformers
from transformers import BertModel, BertConfig, BertTokenizer, AutoTokenizer

In [ ]:
from transformers import AutoModel
model_type = 'bert-base-cased'

config = BertConfig.from_pretrained(model_type, 
output_hidden_states=True, output_attentions=True)  
bert_model = BertModel.from_pretrained(model_type, 
config=config)
# bert_model = AutoModel.from_pretrained('spanbert_hf_base/',config=config) # the path to .bin and config.json

tokenizer = AutoTokenizer.from_pretrained(model_type)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def preprocess(sample_sentence, tokenizer, nlp, candidates=None, pronoun_names=None, pronoun_location=None):
    ######################get candidates#######################
    doc = nlp(sample_sentence)
    if candidates == None:
        candidates = []
        for ent in doc.ents:
            if ent.label_ == "PERSON":
                candidates.append(str(ent.text))
    candidates = list(set(candidates))

    ###########################################################

    #########get candidate locations in tokenized text#########
    tokenized = tokenizer(sample_sentence, return_tensors="pt")
    tokCands = [tokenizer.tokenize(c) for c in candidates]
    begin = 0
    end = 0
    cand_locs = {}
    word_ids = tokenized.word_ids()[1:-1]
    tokens = tokenized.tokens()
    for x in range(len(tokens)):
        for i,c in enumerate(tokCands):
            if tokens[x:x+len(c)] == c:
                if candidates[i] in cand_locs:
                    cand_locs[candidates[i]].append((x,x+len(c)))
                else:
                    cand_locs[candidates[i]]= [(x,x+len(c))]
        # if word_ids[x] == word_ids[x-1]:
        #     end = x
        # else:
        #     tokenS, tokenE = tokenized.word_to_chars(word_ids[begin])
        #     token = sample_sentence[tokenS:tokenE]
        #     if token in candidates:
        #         if token in cand_locs:
        #             cand_locs[token].append((begin+1,end+2))
        #         else:
        #             cand_locs[token] = [(begin+1,end+2)]
        #     begin = end = x
    ############################################################


    ########################get pronouns########################
    if pronoun_names == None:
        pronoun_names = []
        for token in doc:
            if token.pos_ == "PRON":
                pronoun_names.append(str(token))
    pronoun_names = set(pronoun_names)
    pronouns = []
    if pronoun_location == None:
        for x in range(len(word_ids)):
            tokenS, tokenE = tokenized.word_to_chars(word_ids[x])
            token = sample_sentence[tokenS:tokenE]
            if token in pronoun_names:
                pronouns.append(x+1)
    else:
        for x in range(len(word_ids)):
            tokenS, tokenE = tokenized.word_to_chars(word_ids[x])
            if tokenS == pronoun_location:
                pronouns.append(x+1)
                break
    ############################################################

    return tokenized, candidates, cand_locs, pronoun_names, pronouns, tokens

In [ ]:
def coref_bert(model, sentence, candidates, pronouns, pronoun_location, real):
    """
    Harry potter loves to drink butter beer with Hermione and Harry loves to drink, but he loves to eat chicken with Ron.
    Args:
        model: specific pretrained model that we are
            going to use to visualize/analyze attention
        sentence: tokenized sentence that we will analyze
        pronouns: list of indices of pronouns in tokenized sentence
        candidates: dictionary of candidates with list of indices indicating the tokenized spaCy 
            extracted entities that we will assign to each pronoun (there may 
            be more than one candidate mention in the sentence, so be careful)

    Returns:
        A tuple with 2 elements:
            mapping: dict of mapping between pronouns and candidates
            replacement: a string representing the sentence with pronouns 
                replaced with candidate names


    Note: 
        - if there are multiple mentions of the same candidate in 
        the sentence, add the attention between the pronoun and all
        mentions of this candidate.
        - if the candidate consists of multiple subtokens, average the
        attention between the pronoun and all of these subtokens
    """
    tokenized, candidates, cand_locs, pronoun_names, pronouns, tokenized_tokens = preprocess(sentence, tokenizer, nlp, candidates=candidates, pronoun_names=pronouns, pronoun_location=pronoun_location)
    with torch.no_grad():
        outputs = model(**tokenized)
    attentions = outputs[-1]
    attentions = torch.cat(attentions) #(layers, num_heads, seq_len, seq_len)
    attentions = torch.permute(attentions,(2,0,1,3)) #(seq_len, layers, num_heads, seq_len)

    #let's say there are m pronouns and n candidates. We can construct an (m,n) matrix where each row represents how much each pronoun pays attention to the nth candidate
    #for a given layer, head:

    acc = []
    for x in range(12):
        accrow = []
        for y in range(12):
            mat = []
            att = attentions[:,x,y,:]
            for pron_ind in pronouns:
                pron_att = att[pron_ind]
                row = []
                for cand, inds in cand_locs.items():
                    total = 0
                    for start,end in inds:
                        total += torch.mean(pron_att[start:end])
                    row.append(total)
                mat.append(row)
            accrow.append((real == np.argmax(mat[0])))
        acc.append(accrow)
    return np.array(acc)

In [ ]:
# sample_sentence = "Eric and Molly are husband and wife. Unfortunately, Molly hates Eric because he is in love with a different girl. She tries not to cry. But, Eric ends up shooting her in the fucking face."

accuracies = np.zeros((12,12))
for x in tqdm(range(len(df))):
    row = df.iloc[x]
    accuracies += coref_bert(bert_model, row["Text"], [row["A"],row["B"]], [row["Pronoun"]], row["Pronoun-offset"], np.argmax([row["A-coref"],row["B-coref"]]))
accuracies /= len(df)

100%|██████████| 2000/2000 [19:42<00:00,  1.69it/s]


In [ ]:
with open('accuracies.npy', 'wb') as f:
    np.save(f, accuracies)

In [ ]:
np.unravel_index(np.argmax(accuracies, axis=None), accuracies.shape)

(9, 2)

## Experimentation

In [ ]:
attentions = torch.cat(attentions)
attentions.shape #(layers, num_heads, seq_len, seq_len)

torch.Size([12, 12, 29, 29])

In [ ]:
attentions = torch.permute(attentions,(2,0,1,3))
attentions.shape #(seq_len, layers, num_heads, seq_len)

torch.Size([29, 12, 12, 29])

In [ ]:
#iterate through layers and visualize attention
best_he = {}
best_her = {}
she_scores = torch.zeros(attentions.shape[0])
him_scores = torch.zeros(attentions.shape[0])
for l in range(attentions.shape[1]):
    #grab attentions at pos_he and pos_her
    curr_att = attentions[:,l,:,:]
    she_att = curr_att[pos_she,:,:]
    him_att = curr_att[pos_him,:,:]
    he_scores = torch.argmax(she_att,1).squeeze()
    her_scores = torch.argmax(him_att,1).squeeze()
    she_scores += torch.sum(she_att, 0)
    him_scores += torch.sum(him_att, 0)
    for x,y in zip(he_scores, her_scores):
        best_he[tokenized[x]] = best_he.get(tokenized[x],0) + 1
        best_her[tokenized[y]] = best_her.get(tokenized[y],0) + 1
    # now the attentions are (num_heads, seq_len). Each row will be 1 head, so it will be the softmax scores for each of the words.
    print("*"*20)
    print(f"The most attended to word for all heads in layer {l} for 'she' is {[tokenized[r] for r in he_scores]}")
    print(f"The most attended to word for all heads in layer {l} for 'him' is {[tokenized[r] for r in her_scores]}")
    print("*"*20)
    print()

print(tokenized[torch.argmax(attentions[pos_she,8,10,:])])
print(tokenized[torch.argmax(attentions[pos_him,8,10,:])])
print(best_he)
print(best_her)

print(she_scores[correct_she],she_scores[correct_him])
print(him_scores[correct_she],him_scores[correct_him])

********************
The most attended to word for all heads in layer 0 for 'she' is ['tries', 'hates', 'eric', 'because', 'unfortunately', 'girl', 'molly', 'he', 'love', 'eric', 'is', 'hates']
The most attended to word for all heads in layer 0 for 'him' is ['are', 'wife', '.', '.', 'tries', 'girl', 'hates', 'she', 'girl', 'eric', 'tries', 'not']
********************

********************
The most attended to word for all heads in layer 1 for 'she' is ['and', 'is', 'eric', 'eric', 'because', '.', 'eric', 'eric', 'she', 'eric', 'eric', 'he']
The most attended to word for all heads in layer 1 for 'him' is ['eric', 'tries', 'eric', 'eric', '.', '.', 'eric', 'eric', 'he', 'eric', 'eric', 'molly']
********************

********************
The most attended to word for all heads in layer 2 for 'she' is ['is', 'eric', 'eric', 'she', 'eric', 'eric', 'he', 'she', 'eric', 'is', 'hates', 'love']
The most attended to word for all heads in layer 2 for 'him' is ['tries', 'eric', 'eric', 'eric', 'er